In [62]:
import numpy as np
import pandas as pd
from skmultiflow.anomaly_detection import HalfSpaceTrees
from skmultiflow.data.file_stream import FileStream

In [56]:
stream = FileStream('../data/dummy.csv')
stream.prepare_for_use()

print(stream.n_remaining_samples())
print(stream.feature_names)
print(stream.target_names)

1050
['rownum', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5']
['label']


In [57]:

learner = HalfSpaceTrees(n_features=stream.n_features, n_estimators=13, size_limit=75, anomaly_threshold=0.90,depth=10, random_state=5)

cnt = 0
wait_samples = 100
y_preds = []
y_probs = []

while stream.has_more_samples():
    X, y = stream.next_sample()
    if cnt > wait_samples:
        y_pred = learner.predict(X)
        y_prob = learner.predict_proba(X)
        y_preds.append(y_pred)
        y_probs.append(y_prob)
    learner.partial_fit(X, y)
    cnt += 1

In [63]:
pd.DataFrame(y_preds)

,0
0,0
1,0
2,0
3,0
4,0
...,...
944,0
945,0
946,0
947,0


In [64]:
y_proba

[array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
 array([0.5, 0.5]),
